# 验证码识别

验证码

    一种反爬机制。【验证验证码图片中的数据，用于模拟登陆操作】

识别验证码的操作:

- 人工肉眼识别【不推荐】
- 第三方自动识别【推荐】

In [29]:
# 包
import requests
from lxml import etree
import os
# 超级鹰用到的包
from hashlib import md5

In [30]:
# static data

if not os.path.exists('./verCode'):
    os.mkdir('./verCode')

# 登陆页面
url = 'https://so.gushiwen.cn/user/login.aspx?from=http://so.gushiwen.cn/user/collect.aspx'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
}

In [36]:
# crawler
page_text = requests.get(url = url, headers = headers).text
tree = etree.HTML(page_text)
# 验证码图片 src 属性值
src = tree.xpath('//*[@id="imgCode"]/@src')[0]

# 获取验证码的图片
url_code = 'https://so.gushiwen.cn' + src
img = requests.get(url_code, headers = headers).content
# 持久化存储验证码图片【.jpg 的格式居然能用~~~】
# imgName = './verCode/' + src.split('/')[1]
with open('./code.jpg', 'wb') as fp:
    fp.write(img)
print('存储完毕')

存储完毕


In [ ]:
# 用超级鹰解验证码

In [35]:
# 超级鹰封装好的一个类，待会直接 实例化即可
class Chaojiying_Client(object):

    def __init__(self, username, password, soft_id):
        self.username = username
        password =  password.encode('utf8')
        self.password = md5(password).hexdigest()
        self.soft_id = soft_id
        self.base_params = {
            'user': self.username,
            'pass2': self.password,
            'softid': self.soft_id,
        }
        self.headers = {
            'Connection': 'Keep-Alive',
            'User-Agent': 'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0)',
        }

    def PostPic(self, im, codetype):
        """
        im: 图片字节
        codetype: 题目类型 参考 http://www.chaojiying.com/price.html
        """
        params = {
            'codetype': codetype,
        }
        params.update(self.base_params)
        files = {'userfile': ('ccc.jpg', im)}
        r = requests.post('http://upload.chaojiying.net/Upload/Processing.php', data=params, files=files, headers=self.headers)
        return r.json()

    def PostPic_base64(self, base64_str, codetype):
        """
        im: 图片字节
        codetype: 题目类型 参考 http://www.chaojiying.com/price.html
        """
        params = {
            'codetype': codetype,
            'file_base64':base64_str
        }
        params.update(self.base_params)
        r = requests.post('http://upload.chaojiying.net/Upload/Processing.php', data=params, headers=self.headers)
        return r.json()

    def ReportError(self, im_id):
        """
        im_id:报错题目的图片ID
        """
        params = {
            'id': im_id,
        }
        params.update(self.base_params)
        r = requests.post('http://upload.chaojiying.net/Upload/ReportError.php', data=params, headers=self.headers)
        return r.json()

#用户中心>>软件ID 生成一个替换 96001
#本地图片文件路径 来替换 a.jpg 有时WIN系统须要//
#1902 验证码类型  官方网站>>价格体系 3.4+版 print 后要加()
chaojiying = Chaojiying_Client('1940335829', '密码', '938368')	
im = open('./code.jpg', 'rb').read()	
# chaojiying.PostPic(im, 1902) 返回一个字典，pic_str 的值就是验证出来的值
print(chaojiying.PostPic(im, 1902))
#print chaojiying.PostPic(base64_str, 1902)  #此处为传入 base64代码

{'err_no': 0, 'err_str': 'OK', 'pic_id': '1188220240954090001', 'pic_str': '32fo', 'md5': '14f76efa990139365b7e315c12e493e8'}
